In [1]:
import numpy as np
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

device

device(type='cuda', index=0)

In [2]:
import torchvision
from torchvision import models
import torch
from torchvision import transforms
import os
import glob
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [3]:
class SegDataset(Dataset):
    def __init__(self, mask_dir, img_dir):
        self.mask_dir = mask_dir
        self.img_dir = img_dir
        self.n_mask = len(os.listdir(self.mask_dir))
        self.n_img = len(os.listdir(self.img_dir))

    def __len__(self):     
        return min(self.n_img, self.n_mask)

    def __getitem__(self, idx):
        
        filename = os.listdir(self.img_dir)[idx]
        
        img_path = f"{self.img_dir}{filename}"
        img = Image.open(img_path).convert("RGB")
        # image by Height x Width x Channels        
        img = np.asarray(img)/255
        preprocess = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        img = preprocess(img)
        # image dimension changed to Channels x Height x Width
        
        mask_path = f"{self.mask_dir}{filename}"
        mask = Image.open(mask_path).convert('L')
        mask = np.asarray(mask)
        mask[mask>0]=1    # black:0, white: 255
        mask=mask[np.newaxis,:, :]
        mask = torch.Tensor(mask)

        return img, mask

train_mask_dir = f"./images/train/mask/"
train_img_dir = f"./images/train/img/"
training_set = SegDataset(train_mask_dir,train_img_dir)
training_generator = DataLoader(training_set, batch_size=2, shuffle=True)

val_mask_dir = f"./images/validation/mask/"
val_img_dir = f"./images/validation/img/"
validation_set = SegDataset(val_mask_dir, val_img_dir)
validation_generator = DataLoader(validation_set, batch_size=2, shuffle=True)
    

In [4]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models


def createDeepLabv3(outputchannels=1):
    """DeepLabv3 class with custom head
    Args:
        outputchannels (int, optional): The number of output channels
        in your dataset masks. Defaults to 1.
    Returns:
        model: Returns the DeepLabv3 model with the ResNet101 backbone.
    """
    model = models.segmentation.deeplabv3_resnet50(pretrained=True,
                                                    progress=True)
    model.classifier = DeepLabHead(2048, outputchannels)
    # Set the model in training mode
    model.train()
    return model

model = createDeepLabv3()

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to C:\Users\z1061/.cache\torch\hub\checkpoints\deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

In [5]:
from sklearn.metrics import f1_score, roc_auc_score

# Specify the loss function
criterion = torch.nn.MSELoss(reduction='mean')
# Specify the optimizer with a lower learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Specify the evaluation metrics
metrics = {'f1_score': f1_score, 'auroc': roc_auc_score}

In [6]:
model.to(device)
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    total_loss = 0
    optimizer.zero_grad()
    for (img, mask) in iter(training_generator):
        img = img.to(device).to(torch.float32)
        mask = mask.to(device).to(torch.float32)
        output = model(img)
        loss = criterion(output['out'], mask)
        loss.backward()
        optimizer.step()
        total_loss += loss
    print(f"Epoch {epoch}: {total_loss:.4f}")

C:\Users\z1061\Anaconda3\envs\pytorchenv_gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 1: 14.9687
Epoch 2: 21.8872
Epoch 3: 12.2049
Epoch 4: 8.0692


KeyboardInterrupt: 

In [12]:
for (img, mask) in iter(validation_generator):
    img = img.to(device).to(torch.float32)
    mask = mask.to(device).to(torch.float32)
    output = model(img).detach()
    out = output["out"]
    break
print(output)

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 8.00 GiB total capacity; 6.53 GiB already allocated; 0 bytes free; 6.76 GiB reserved in total by PyTorch)

In [7]:
import copy
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 1e10
model.to(device)

num_epochs = 5

for epoch in range(1, num_epochs + 1):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print('-' * 10)
    
    train_epoch_loss = 0
    test_epoch_loss = 0
    
    for phase in ['Train', 'Test']:
        if phase == 'Train':
            model.train()  # Set model to training mode
            for sample in iter(training_generator):
                inputs = sample[0].to(device)
                inputs = inputs.to(torch.float32)
                masks = sample[1].to(device)
                masks = masks.to(torch.float32)

                optimizer.zero_grad()
#                 print(inputs.size(), masks.size())
#                 print(inputs.dtype, masks.dtype)
                
                outputs = model(inputs)
                loss = criterion(outputs['out'], masks)
                batch_loss = loss.item()
                
#                 y_pred = outputs['out'].data.cpu().numpy().ravel()
#                 y_true = masks.data.cpu().numpy().ravel()
#                 train_f1.append(f1_score(y_true > 0, y_pred > 0.1))
#                 train_auroc.append(roc_auc_score(y_true.astype('uint8'), y_pred))

                loss.backward()
                optimizer.step()
                train_epoch_loss += batch_loss
                print('{} Batch Loss: {:.4f}'.format(phase, batch_loss))
        
        else:
            model.eval()  # Set model to evaluate mode
            for sample in iter(validation_generator):
                inputs = sample[0].to(device)
                inputs = inputs.to(torch.float32)
                masks = sample[1].to(device)
                masks = masks.to(torch.float32)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs['out'], masks)
                batch_loss = loss.item()
                test_epoch_loss += batch_loss

#                 y_pred = outputs['out'].data.cpu().numpy().ravel()
#                 y_true = masks.data.cpu().numpy().ravel()
#                 test_f1.append(f1_score(y_true > 0, y_pred > 0.1))
#                 test_auroc.append(roc_auc_score(y_true.astype('uint8'), y_pred))
                print('{} Batch Loss: {:.4f}'.format(phase, batch_loss))
            
    print(f"Epoch {epoch}/{num_epochs} | Train Loss: {train_epoch_loss:.4f} | Test Loss: {test_epoch_loss:.4f}")



Epoch 1/5
----------


C:\Users\z1061\Anaconda3\envs\pytorchenv_gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Batch Loss: 0.8659
Train Batch Loss: 0.9845
Train Batch Loss: 0.6883
Train Batch Loss: 0.5938
Train Batch Loss: 0.8225
Train Batch Loss: 0.5154
Train Batch Loss: 0.4433
Train Batch Loss: 0.5129
Train Batch Loss: 0.3783
Train Batch Loss: 0.9717
Train Batch Loss: 1.0439
Train Batch Loss: 0.8207
Train Batch Loss: 0.9097
Train Batch Loss: 0.6193
Train Batch Loss: 0.4627
Train Batch Loss: 0.3661
Train Batch Loss: 0.6654
Train Batch Loss: 0.3967
Train Batch Loss: 0.2922
Train Batch Loss: 0.2790
Train Batch Loss: 0.2386
Train Batch Loss: 0.1873
Train Batch Loss: 0.1562
Train Batch Loss: 0.2110
Train Batch Loss: 0.1973
Test Batch Loss: 0.3617


RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 8.00 GiB total capacity; 6.49 GiB already allocated; 0 bytes free; 6.74 GiB reserved in total by PyTorch)